In [1]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import *
from filters import resamplers
from filters.filter import ParticleFilter
from filters.filter import ParticleFilterAll
jax.config.update("jax_enable_x64", True)
import numpy as np

Initialisation of a twin experiment. 

In [2]:
signal_params = ConfigDict(KS_params_SALT)
ensemble_params = ConfigDict(KS_params_SALT)
ensemble_params.update(E=1,P=32,stochastic_advection_basis='constant')
ensemble_params.update(E=128,P=32,stochastic_advection_basis='constant')

Now we specify the models, by calling the class.

In [3]:
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)

I0000 00:00:1740834485.928580       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


multinomial, systematic, no_resampling, default


In [4]:
observation_spatial_frequency = 16
observation_locations = np.arange(0,signal_model.x.shape[0],observation_spatial_frequency)
observation_noise = 0.1
number_of_observations_time =ensemble_model.nt
observation_temporal_frequency = int(ensemble_model.nt/number_of_observations_time)

print(observation_locations)
pf_systematic = ParticleFilterAll(
    n_particles = ensemble_params.E,
    n_steps = observation_temporal_frequency,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    seed = 11,
    resampling="systematic",#'default',
    observation_locations = observation_locations,
)

[  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240]


In [5]:
da_steps = number_of_observations_time
final, all = pf_systematic.run(initial_ensemble, initial_signal, da_steps) #the final input is scan length? 

IndexError: Too many indices for array: 3 non-None/Ellipsis indices for dim 2.

Put in the initial condition.

In [ ]:
particles =jnp.concatenate([initial_ensemble[None,...], all[0]], axis=0)
signal = jnp.concatenate([initial_signal[None,...], all[1]], axis=0)
observations = jnp.concatenate([initial_signal[None,...], all[2]], axis=0)
observations = all[2][:,:, observation_locations]
print(observations.shape)
print(particles.shape)

In [ ]:
def plot(da_step):
    plt.plot(signal_model.x, signal[da_step,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[da_step,:,:].T, color='b',label='particles',linewidth=0.1)
    if da_step > 0:
        plt.plot(signal_model.x[observation_locations], observations[da_step-1,0,:], 'ro',label='observations')
    plt.show()

interact(plot, da_step=(0, da_steps))